In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_dir = '/content/drive/MyDrive/IndoorOutdoorDataset'

In [ ]:
img_width, img_height = 299, 299
batch_size = 32
epochs = 10

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='training',
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='validation',
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=[Accuracy()])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                        

In [ ]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
)

Epoch 1/10
20/20 [==============================] - 292s 13s/step - loss: 2.9812 - accuracy: 0.0719 - val_loss: 1.7310 - val_accuracy: 0.0250
Epoch 2/10
20/20 [==============================] - 256s 13s/step - loss: 0.8248 - accuracy: 0.0000e+00 - val_loss: 0.6240 - val_accuracy: 0.0000e+00
Epoch 3/10
20/20 [==============================] - 204s 10s/step - loss: 0.4580 - accuracy: 0.0000e+00 - val_loss: 0.5296 - val_accuracy: 0.0000e+00
Epoch 4/10
20/20 [==============================] - 205s 10s/step - loss: 0.4912 - accuracy: 0.0000e+00 - val_loss: 0.5857 - val_accuracy: 0.0000e+00
Epoch 5/10
20/20 [==============================] - 204s 10s/step - loss: 0.8014 - accuracy: 0.0000e+00 - val_loss: 0.6753 - val_accuracy: 0.0000e+00
Epoch 6/10
20/20 [==============================] - 205s 10s/step - loss: 0.5276 - accuracy: 0.0000e+00 - val_loss: 0.5775 - val_accuracy: 0.0000e+00
Epoch 7/10
20/20 [==============================] - 206s 10s/step - loss: 0.4494 - accuracy: 0.0000e+00 - va

In [ ]:
model.save('/content/drive/MyDrive/inceptionv3_indoor_outdoor_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
